In [1]:
%env DATA_DIR=$HOME/datasets
%env EXPERIMENT_BASE=$HOME/experiments/ood_flows
%env LOG_LEVEL=INFO
%env BATCH_SIZE=64
%env OPTIM_LR=0.001
%env OPTIM_M=0.8
%env TRAIN_EPOCHS=100
%env EXC_RESUME=1
%env MANIFOLD_D=1024
%env MODEL_NAME=fisher_exact_ce
%env DATASET_NAME=QPM_species_resnet50_simclr_M128
# %env OOD=4

env: DATA_DIR=$HOME/datasets
env: EXPERIMENT_BASE=$HOME/experiments/ood_flows
env: LOG_LEVEL=INFO
env: BATCH_SIZE=64
env: OPTIM_LR=0.001
env: OPTIM_M=0.8
env: TRAIN_EPOCHS=100
env: EXC_RESUME=1
env: MANIFOLD_D=1024
env: MODEL_NAME=fisher_exact_ce
env: DATASET_NAME=QPM_species_resnet50_simclr_M128


In [2]:
import lightning.pytorch as pl
import numpy as np
import torch
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers.wandb import WandbLogger

from config import load_config

In [3]:
# initialize the RNG deterministically
np.random.seed(42)
torch.manual_seed(42)
torch.set_float32_matmul_precision('medium')

config = load_config()

In [4]:
emb_path = f"assets/embeddings/{config.dataset_name}.npz"
config.load_embedding(embedding_path=emb_path, num_dims=config.manifold_d, num_targets=21)

/n/home12/yasith/.conda/envs/qpm-amrb/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model = config.get_model(model_name=config.model_name, in_dims=config.manifold_d, rand_dims=1000)

In [6]:
run_name = f"{config.dataset_name}_{config.model_name}"
wandb_logger = WandbLogger(project="uq_project", log_model="all", name=run_name, config={**config.params, **model.hparams})
wandb_logger.watch(model, log="all")

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yasith. Use `wandb login --relogin` to force relogin


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


In [ ]:
checkpoint_callback = ModelCheckpoint(monitor="val_accuracy", mode="max")
trainer = pl.Trainer(
    logger=wandb_logger,
    max_epochs=config.train_epochs,
    callbacks=[checkpoint_callback],
    reload_dataloaders_every_n_epochs=5,
)

trainer.fit(model=model, datamodule=config.datamodule)

/n/home12/yasith/.conda/envs/qpm-amrb/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:168: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /n/home12/yasith/.conda/envs/qpm-amrb/lib/python3.10 ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/n/home12/yasith/.conda/envs/qpm-amrb/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:168: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /n/home12/yasith/.conda/envs/qpm-amrb/lib/python3.10 ...
  rank_zero_warn(
Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2
/n/hom

Epoch 0: 100%|██████████| 229/229 [00:10<00:00, 21.55it/s, v_num=lqvj, train_loss_y_true=2.840, train_loss_y_rand=2.990, train_loss=5.830]
Validation: 0it [00:00, ?it/s]
Epoch 1: 100%|██████████| 229/229 [00:09<00:00, 23.59it/s, v_num=lqvj, train_loss_y_true=2.870, train_loss_y_rand=2.800, train_loss=5.670, val_loss_y_true=2.980, val_loss_y_rand=3.020, val_loss=5.990]
Validation: 0it [00:00, ?it/s]
Epoch 2: 100%|██████████| 229/229 [00:09<00:00, 23.38it/s, v_num=lqvj, train_loss_y_true=2.950, train_loss_y_rand=2.780, train_loss=5.730, val_loss_y_true=2.780, val_loss_y_rand=2.800, val_loss=5.580]
Validation: 0it [00:00, ?it/s]
Epoch 2: 100%|██████████| 229/229 [00:13<00:00, 16.91it/s, v_num=lqvj, train_loss_y_true=2.950, train_loss_y_rand=2.780, train_loss=5.730, val_loss_y_true=2.650, val_loss_y_rand=2.670, val_loss=5.320]